# Table Merges

## Imports

In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

In [2]:
# DATA
## ADV - Changed to own rel. paths
aircraft_df = pd.read_csv('../raw_data/csv/aircraft.csv')
events_df = pd.read_csv('../raw_data/csv/events.csv')
flight_crew_df = pd.read_csv('../raw_data/csv/Flight_Crew.csv')
flight_time_df = pd.read_csv('../raw_data/csv/flight_time.csv')
engines_df = pd.read_csv('../raw_data/trimmed_data/engines_fin.csv')
events_sequence_df = pd.read_csv('../raw_data/trimmed_data/Events_Sequence_fin.csv')

/tmp/ipykernel_27513/2276705612.py:3: DtypeWarning: Columns (0,59) have mixed types. Specify dtype option on import or set low_memory=False.
  aircraft_df = pd.read_csv('../raw_data/csv/aircraft.csv')
/tmp/ipykernel_27513/2276705612.py:4: DtypeWarning: Columns (0,31,51) have mixed types. Specify dtype option on import or set low_memory=False.
  events_df = pd.read_csv('../raw_data/csv/events.csv')


## Flight time

In [3]:
mask = flight_time_df['flight_craft'] == 'ALL' 
flight_time_df = flight_time_df[mask]

mask_2 = flight_time_df['flight_type'] == 'TOTL'
flight_time_df = flight_time_df[mask_2]

flight_time_df['id'] = (flight_time_df['ev_id'].astype(str) + flight_time_df['Aircraft_Key'].astype(str)).astype('object')
flight_time_df.drop(['ev_id', 'Aircraft_Key'], axis=1, inplace=True)

flight_time_df

,crew_no,flight_type,flight_craft,flight_hours,lchg_date,lchg_userid,id
12,1,TOTL,ALL,705.0,2020-09-25 18:05:31,NaN,20080107X000261
34,1,TOTL,ALL,8300.0,2020-09-25 18:05:31,NaN,20080107X000262
58,1,TOTL,ALL,1000.0,2020-09-25 18:05:31,NaN,20080109X000361
64,1,TOTL,ALL,10571.0,2020-09-25 18:04:02,NaN,20080107X000271
75,1,TOTL,ALL,18.0,2020-09-25 18:06:16,NaN,20080115X000511
...,...,...,...,...,...,...,...
327105,2,TOTL,ALL,122.0,2023-04-20 18:02:40,coln,202303131068891
327122,1,TOTL,ALL,74.0,2023-04-27 18:00:03,coln,202303201069201
327160,1,TOTL,ALL,23000.0,2023-04-27 18:00:03,coln,202303241069471
327171,1,TOTL,ALL,33100.0,2023-05-04 18:00:11,fina,202303311069871


In [4]:
flight_time_df.head()

,crew_no,flight_type,flight_craft,flight_hours,lchg_date,lchg_userid,id
12,1,TOTL,ALL,705.0,2020-09-25 18:05:31,NaN,20080107X000261
34,1,TOTL,ALL,8300.0,2020-09-25 18:05:31,NaN,20080107X000262
58,1,TOTL,ALL,1000.0,2020-09-25 18:05:31,NaN,20080109X000361
64,1,TOTL,ALL,10571.0,2020-09-25 18:04:02,NaN,20080107X000271
75,1,TOTL,ALL,18.0,2020-09-25 18:06:16,NaN,20080115X000511


In [5]:
flight_time_df.set_index('id', inplace=True)
flight_hours_mean = pd.DataFrame(flight_time_df['flight_hours'].groupby('id').mean())
flight_hours_mean.columns = ['flight_hours_mean']

In [6]:
flight_time_df = flight_time_df[['flight_type', 'flight_craft']]
flight_time_df.head()

,flight_type,flight_craft
id,,
20080107X000261,TOTL,ALL
20080107X000262,TOTL,ALL
20080109X000361,TOTL,ALL
20080107X000271,TOTL,ALL
20080115X000511,TOTL,ALL


In [7]:
flight_hours_mean

,flight_hours_mean
id,
20080107X000261,705.0
20080107X000262,8300.0
20080107X000271,10571.0
20080109X000361,1000.0
20080114X000441,537.0
...,...
202303131068891,122.0
202303201069201,74.0
202303241069471,23000.0


In [8]:
flight_time_mean = flight_hours_mean.join(flight_time_df, how='inner')
flight_time_mean = flight_time_mean[~flight_time_mean.index.duplicated(keep='first')]

In [9]:
# ADV - Dropped filtered columns ('flight_type' & 'flight_craft')
flight_time_mean.drop(['flight_type', 'flight_craft'], axis=1, inplace=True)

In [10]:
flight_time_mean.shape

(19552, 1)

## Flight crew

In [11]:
# ADV - Added 'crew_no' to enable boolean mask
flight_crew_df = flight_crew_df[['ev_id', 'Aircraft_Key', 'crew_no', 'crew_category', 'crew_sex', 'pc_profession', 'crew_tox_perf', 'mr_faa_med_certf', 'available_restraint']]

In [12]:
# ADV - Changed mask to return unique rows & more categories for 'crew_category'
mask = (flight_crew_df['crew_category'].isin(['PLT','DSTU','FLTI','CPLT','KPLT'])) & (flight_crew_df['crew_no'] == 1)
flight_crew_df = flight_crew_df[mask]

flight_crew_df['id'] = (flight_crew_df['ev_id'].astype(str) + flight_crew_df['Aircraft_Key'].astype(str)).astype('object')
flight_crew_df.drop(['ev_id', 'Aircraft_Key'], axis=1, inplace=True)

In [13]:
# ADV - Dropped filtered columns ('crew_no')
flight_crew_df.drop(['crew_no'], axis=1, inplace=True)

In [14]:
flight_crew_df.set_index('id', inplace=True)
flight_crew_df.head()

,crew_category,crew_sex,pc_profession,crew_tox_perf,mr_faa_med_certf,available_restraint
id,,,,,,
20080107X000261,PLT,M,No,N,NaN,NaN
20080107X000262,PLT,M,No,N,NaN,NaN
20080109X000361,PLT,F,No,N,NaN,NaN
20080107X000271,PLT,M,UNK,N,NaN,NaN
20080115X000511,DSTU,M,No,N,NaN,NaN


In [15]:
flight_crew_df.shape

(20693, 6)

In [16]:
flight_time_crew = flight_time_mean.join(flight_crew_df, how='inner')
flight_time_crew.shape

(19474, 7)

## Aircraft

In [17]:
aircraft_df = aircraft_df[['ev_id', 'Aircraft_Key', 'far_part', 'flt_plan_filed', 'acft_make',  'acft_model', 'total_seats', 'num_eng', 'fixed_retractable', 'cert_max_gr_wt', 'acft_category', 'homebuilt', 'type_last_insp', 'afm_hrs_last_insp', 'afm_hrs', 'elt_install', 'elt_type', 'oper_dba', 'certs_held', 'type_fly', 'second_pilot', 'dprt_apt_id', 'dprt_time', 'dest_apt_id', 'site_seeing', 'air_medical']]

aircraft_df['id'] = (aircraft_df['ev_id'].astype(str) + aircraft_df['Aircraft_Key'].astype(str)).astype('object')
aircraft_df.drop(['ev_id', 'Aircraft_Key'], axis=1, inplace=True)

In [18]:
aircraft_df.set_index('id', inplace=True)
aircraft_df.head()

,far_part,flt_plan_filed,acft_make,acft_model,total_seats,num_eng,fixed_retractable,cert_max_gr_wt,acft_category,homebuilt,...,elt_type,oper_dba,certs_held,type_fly,second_pilot,dprt_apt_id,dprt_time,dest_apt_id,site_seeing,air_medical
id,,,,,,,,,,,,,,,,,,,,,
20080211X001751,NUSN,NaN,Eurocopter France,AS350B3,NaN,NaN,FIXD,NaN,HELI,N,...,NaN,NaN,NaN,EXLD,NaN,NaN,NaN,NaN,N,N
20080107X000261,091,NONE,PIPER,PA 28-180,4.0,1.0,FIXD,2400.0,AIR,N,...,NaN,NaN,Y,PERS,N,O69,2200.0,OQ3,N,N
20080107X000262,091,NONE,Barnard/Stancil,Glastar,2.0,1.0,FIXD,1960.0,AIR,Y,...,NaN,NaN,Y,PERS,N,OQ3,2230.0,O70,N,N
20080109X000361,091,NONE,Micco Aircraft Company,MAC-145B,2.0,1.0,RETR,2850.0,AIR,N,...,NaN,NaN,Y,PERS,N,T20,2100.0,AXH,N,N
20080107X000271,091,IFR,Pilatus,PC-12/45,9.0,1.0,RETR,9920.0,AIR,N,...,NaN,NaN,Y,PERS,N,PHX,400.0,PWA,N,N


In [19]:
# ADV - Checking shape
aircraft_df.shape

(26064, 24)

In [20]:
aircraft_fc_ft = aircraft_df.join(flight_time_crew, how='inner')
aircraft_fc_ft.shape

(19474, 31)

## Engines

In [21]:
engines_df.shape

(23669, 14)

In [22]:
engines_df.drop(columns=["Unnamed: 0", "eng_time_total", "eng_time_last_insp", "eng_time_overhaul"], inplace=True)

mask = engines_df['eng_no'] == 1 
engines_df = engines_df[mask]

engines_df['id'] = (engines_df['ev_id'].astype(str) + engines_df['Aircraft_Key'].astype(str)).astype('object')
engines_df.drop(['ev_id', 'Aircraft_Key'], axis=1, inplace=True)
engines_df.set_index('id', inplace=True)

In [23]:
engines_df.shape

(20876, 8)

## Wingman X and y

In [24]:
X = aircraft_fc_ft.join(engines_df, how='inner')

In [25]:
X.shape

(18893, 39)

In [26]:
# ADV - Added code on line 4 to drop 'phase_no' column from target table

y = events_sequence_df.copy()
y.drop(['phase_no'], axis=1, inplace=True)
y.set_index('id', inplace=True)

In [27]:
# ADV - Commented out code intended to remove duplicates

wingman_data = y.join(X, how='inner')
# wingman_data = wingman_data[~wingman_data.index.duplicated(keep='first')]

In [28]:
# ADV - Code to return df of duplicated index entries (i.e check none exist)
wingman_data[wingman_data.index.to_series().duplicated(keep=False)]

,eventsoe_no,far_part,flt_plan_filed,acft_make,acft_model,total_seats,num_eng,fixed_retractable,cert_max_gr_wt,acft_category,...,mr_faa_med_certf,available_restraint,eng_no,eng_type,eng_mfgr,eng_model,power_units,hp_or_lbs,carb_fuel_injection,propeller_type
id,,,,,,,,,,,,,,,,,,,,,


In [29]:
wingman_data.shape

(18811, 40)

In [30]:
wingman_data.shape, events_sequence_df.shape

((18811, 40), (25154, 3))

In [31]:
# ADV - Dropped 'phase_no' column from events_sequence_df
 
events_sequence_df.set_index('id', inplace=True)
events_sequence_df.drop(['phase_no'], axis=1, inplace=True)
events_sequence_df.head()

,eventsoe_no
id,
20080107X000261,250
20080107X000262,250
20080109X000361,230
20080107X000271,81
20080115X000511,240


In [32]:
wingman_data_indices = list(wingman_data.index)
events_sequence_indices = list(events_sequence_df.index)

In [33]:
missing_indices = [i for i in events_sequence_indices if i not in wingman_data_indices]

In [34]:
# ADV - Code returning sum of missing indices
len(missing_indices)

6343

In [35]:
# ADV - Write to CSV

import os
 
output_directory = r"/home/aaron/code/AaronDV17/wingman-project/raw_data/trimmed_data"

output_file = os.path.join(output_directory, "wingman_data.csv")

pd.DataFrame.to_csv(wingman_data, output_file)